In [200]:
!pip install elasticsearch

In [201]:
!pip install gdown

## Setting up the Elasticsearch

In [202]:
from datetime import datetime
from elasticsearch import Elasticsearch

ELASTIC_PASSWORD = "p2iFCHUbC7ze1QoIMVw"

es = Elasticsearch("http://elasticsearch:9200",
                    basic_auth=("elastic", ELASTIC_PASSWORD))

es.info()

ObjectApiResponse({'name': 'es-node', 'cluster_name': 'tdt4117-ir-data-cluster', 'cluster_uuid': 'rAkxpaFNSfypbZhfl9rXuA', 'version': {'number': '8.4.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '89f8c6d8429db93b816403ee75e5c270b43a940a', 'build_date': '2022-09-14T16:26:04.382547801Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

## Setting up the documents

In [203]:
# Downloading the data
"""
The data is uploaded at a google drive directory, by running this code box, 
the data will be downloaded and you will have access to it at
(enron_short/maildir)
"""
# import gdown

# googleDriveURL = "https://drive.google.com/file/d/10OwgK91e0lNRsrAV31sSJu7KJUujBTnE/view?usp=sharing"
# output = 'enron_short.tar.gz'
# gdown.download(googleDriveURL, output, quiet=False, fuzzy=True)

'\nThe data is uploaded at a google drive directory, by running this code box, \nthe data will be downloaded and you will have access to it at\n(enron_short/maildir)\n'

In [204]:
# check if the directory exists
from os import path

zipfile_path = "enron_short.tar.gz"
if not path.isfile(zipfile_path):
    print("the zipfile is not here, please ensure you download it first.")

In [205]:
!tar xf enron_short.tar.gz

In [206]:
# check if the directory exists
from os import path

documents_path = r"enron_short/maildir"
if not path.exists(documents_path):
    print("the directory is not here, please ensure you have the documnets first.")

## Your Code:

In [224]:
# Index all emails folder enron_short/maildir
es.indices.delete(index="index", ignore_unavailable=True) # Delete the index if it exists
# Create the index with mapping for email body
es.indices.create(index='index', body={
    "mappings": {
        "properties": {
            "body": {
                "type": "text"
            }
        }
    }
})

import os
from tqdm import tqdm

i = 1
for root, dirs, files in tqdm(os.walk(documents_path)):
    for file in files:
        with open(os.path.join(root, file), "r", encoding="utf-8", errors="ignore") as f:
            data = f.read()
            es.index(index="index", id=i, body={"content": data})
            i += 1

es.get(index="index", id=1)

192821it [25:36, 125.48it/s]


ObjectApiResponse({'_index': 'index', '_id': '1', '_version': 1, '_seq_no': 0, '_primary_term': 1, '_ignored': ['content.keyword'], 'found': True, '_source': {'content': 'Message-ID: <1706910.1075842665022.JavaMail.evans@thyme>\nDate: Wed, 16 Jun 1999 08:32:00 -0700 (PDT)\nFrom: bieraugel@efortress.com\nTo: gerald.nemec@enron.com\nSubject: Catching up\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: "Paul Bieraugel" <bieraugel@efortress.com>\nX-To: Gerald Nemec\nX-cc: \nX-bcc: \nX-Folder: \\Gerald_Nemec_Dec2000_June2001_1\\Notes Folders\\Personal\nX-Origin: NEMEC-G\nX-FileName: gnemec.nsf\n\nGerald,  good to hear from you and very belated congrats on your law\ndegree/bar exam/ascent to layerhood deal.  That\'s way cool.  Sounds like you\nare really doing well out there.  Tool belts, house restoration, sounds like\nfun.  Too bad the Navy won\'t allow me the time to do it.   Regardless,\nthey\'ve been moving me around so much lately

In [230]:
def pretty_print(query, res):
    print("Query: ", query)
    print("Number of results: ", res["hits"]["total"]["value"])
    print("Results: ")
    for hit in res["hits"]["hits"]:
        print("Score: ", hit["_score"])
        print("Content: ", hit["_source"]["content"][:100] + "...")

In [234]:
# Query for "Norwegian and University and Science and Technology” and "Norwegian University Science Technology"

query = "Norwegian and University and Science and Technology"
res = es.search(index="index", body={"query": {"match": {"content": query}}})
pretty_print(query, res)

Query:  Norwegian and University and Science and Technology
Number of results:  10000
Results: 
Score:  22.428768
Content:  Message-ID: <21996921.1075855468318.JavaMail.evans@thyme>
Date: Fri, 21 Dec 2001 04:43:38 -0800 (PST...
Score:  22.428768
Content:  Message-ID: <24164547.1075840774929.JavaMail.evans@thyme>
Date: Fri, 21 Dec 2001 04:43:38 -0800 (PST...
Score:  21.451519
Content:  Message-ID: <33145748.1075855468410.JavaMail.evans@thyme>
Date: Sun, 23 Dec 2001 14:45:10 -0800 (PST...
Score:  21.451519
Content:  Message-ID: <21179930.1075840774807.JavaMail.evans@thyme>
Date: Sun, 23 Dec 2001 14:45:10 -0800 (PST...
Score:  21.329004
Content:  Message-ID: <24069195.1075846942557.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 11:26:00 -0700 (PDT...
Score:  21.329004
Content:  Message-ID: <9729632.1075847069332.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 11:26:00 -0700 (PDT)...
Score:  21.329004
Content:  Message-ID: <4007612.1075855889171.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 11:2

In [233]:
query = "Norwegian University Science Technology"
res = es.search(index="index", body={"query": {"match": {"content": query}}})
pretty_print(query, res)

Query:  Norwegian University Science Technology
Number of results:  9744
Results: 
Score:  20.404533
Content:  Message-ID: <21996921.1075855468318.JavaMail.evans@thyme>
Date: Fri, 21 Dec 2001 04:43:38 -0800 (PST...
Score:  20.404533
Content:  Message-ID: <24164547.1075840774929.JavaMail.evans@thyme>
Date: Fri, 21 Dec 2001 04:43:38 -0800 (PST...
Score:  19.514381
Content:  Message-ID: <33145748.1075855468410.JavaMail.evans@thyme>
Date: Sun, 23 Dec 2001 14:45:10 -0800 (PST...
Score:  19.514381
Content:  Message-ID: <21179930.1075840774807.JavaMail.evans@thyme>
Date: Sun, 23 Dec 2001 14:45:10 -0800 (PST...
Score:  18.755571
Content:  Message-ID: <24069195.1075846942557.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 11:26:00 -0700 (PDT...
Score:  18.755571
Content:  Message-ID: <9729632.1075847069332.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 11:26:00 -0700 (PDT)...
Score:  18.755571
Content:  Message-ID: <4007612.1075855889171.JavaMail.evans@thyme>
Date: Mon, 24 Apr 2000 11:26:00 -0700 (P